In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

pass_list = os.listdir('/content/drive/MyDrive/DeepLearning/Project/pass')
fail_list = os.listdir('/content/drive/MyDrive/DeepLearning/Project/fail')

train_X = []
for img in pass_list:
    
    # 이미지 로드 및 크기 조정
    image = cv2.imread(f"/content/drive/MyDrive/DeepLearning/Project/pass/{img}")
    imageHeight, imageWidth, _ = image.shape

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
    image_resized = cv2.resize(image, (75,100), interpolation=cv2.INTER_AREA)
    
    # 크기 조정된 이미지를 train_X 배열에 추가
    train_X.append(image_resized)

for img in fail_list:
    
    # 이미지 로드 및 크기 조정
    image = cv2.imread(f"/content/drive/MyDrive/DeepLearning/Project/fail/{img}")
    imageHeight, imageWidth, _ = image.shape

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
    image_resized = cv2.resize(image, (75,100), interpolation=cv2.INTER_AREA)
    
    # 크기 조정된 이미지를 train_X 배열에 추가
    train_X.append(image_resized)

# train_X 배열을 numpy 배열로 변환
train_X = np.array(train_X)

# train_X 배열의 형상(shape) 확인
print(train_X.shape)

# 정답지
train_y = np.ones(len(pass_list))
train_y = np.append(train_y,np.zeros(len(fail_list)))
train_y

train_X = train_X/255.0

(3648, 100, 75, 3)


In [ ]:
import numpy as np
from sklearn.utils import shuffle

# 데이터 셔플
shuffled_train_X, shuffled_train_y = shuffle(train_X, train_y)

# 결과 확인
print(shuffled_train_X.shape)
print(shuffled_train_y.shape)

(3648, 100, 75, 3)
(3648,)


In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D,Flatten,Dense, Dropout

In [ ]:
model = tf.keras.Sequential([
    # 입력은 1장 : 28 28 1 =>
    # 1번 layer : conv ==> size, 몇 개
    tf.keras.layers.Conv2D(input_shape = (100,75,3), kernel_size = (3,3), filters = 16,
                    ),
    # 2번 layer : conv
    tf.keras.layers.Conv2D( kernel_size = (3,3),filters = 32),
    # 3번 layer : conv
    tf.keras.layers.Conv2D( kernel_size = (3,3), filters = 64),
    ### --> 이미지들에서 특징들을 추출했다.. 

    ### step2 : 분류에 대한 DNN
    tf.keras.layers.Flatten( ),# 중간에 낀 Layer기 때문에 아무것도 안해도 상관없음

    # 1개만 layer를 추가하겠습니다.
    tf.keras.layers.Dense(units = 128, activation = 'relu'),

    # 아웃풋 레이어 --> 맞춰야 하는 부분
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = 2, activation = 'softmax' )

])
# loss를 sparse
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"]
              )
history = model.fit(train_X,train_y,validation_split=0.2,
          batch_size = 128, epochs = 100)

AttributeError: ignored

In [ ]:
result = model.predict("/content/test_1.jpg")

IndexError: ignored

In [ ]:
image = cv2.imread(f"test_6.jpg")
imageHeight, imageWidth, _ = image.shape

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
image_resized = cv2.resize(image, (75,100), interpolation=cv2.INTER_AREA)\

test = image_resized/255.0
test = test.reshape(-1,100,75,3)

result = model.predict(test)
result

In [ ]:
result = model.predict(test)
result

1/1 [==============================] - 0s 231ms/step


array([[0.982602  , 0.01739802]], dtype=float32)

In [ ]:
# 특징 추출기로 : Inception V3  
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
train_data_sub_folder = "/content/drive/MyDrive/DeepLearning/Project/image_classification/"
batch_size = 32
image_height = 200
image_width = 150

In [ ]:
inception_url = "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5"
feature_model = tf.keras.Sequential(
    [
        hub.KerasLayer(inception_url, output_shape=(1536,),
                       trainable=False)
    ]
)
feature_model.build( [ None, 299,299,3])
feature_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1536)              54336736  
                                                                 
Total params: 54,336,736
Trainable params: 0
Non-trainable params: 54,336,736
_________________________________________________________________


In [ ]:
# 이미지생성용으로...
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    validation_split = 0.25,
    height_shift_range=0.08, zoom_range=0.08
)
valid_datagen = ImageDataGenerator(
    rescale = 1./255.,
    validation_split = 0.25,
    height_shift_range=0.08, zoom_range=0.08
)
train_generator = train_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "training",
    batch_size = 128,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)
valid_generator = valid_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "validation",
    batch_size = 32,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)

Found 1846 images belonging to 2 classes.
Found 615 images belonging to 2 classes.


In [ ]:
import numpy as np
batch_step = int(1846*1) // 128
# train 데이터를 특징을 추출한 것을 메모리에 담으려고 함
train_features = []
train_y = []
# 너무 많아서 줄이기

# for idx in range(20):
for idx in range(batch_step):
  if idx % 200 == 0:
    print(idx)
  x,y = train_generator.next() # 데이터 돌리기..
  train_y.extend(y) # 정답지 추가
  features = feature_model.predict(x)
  train_features.extend(features)
train_features = np.array(train_features)
train_y = np.array(train_y)
print(train_features.shape)

0
4/4 [==============================] - 0s 101ms/step
(1792, 1536)


In [ ]:
batch_step = int(615 *1) // 32
# train 데이터를 특징을 추출한 것을 메모리에 담으려고 함
valid_features = []
valid_y = []

# for idx in range(5):
for idx in range(batch_step):
  if idx % 200 == 0:
    print(idx)
  x,y = valid_generator.next() # 데이터 돌리기..
  valid_y.extend(y) # 정답지 추가
  features = feature_model.predict(x)
  valid_features.extend(features)
valid_features = np.array(valid_features)
valid_y = np.array(valid_y)
print(valid_features.shape)

0
1/1 [==============================] - 0s 46ms/step
(608, 1536)


In [ ]:
# 가볍게 분류 모델을 작성할 수 있음 !!!
model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(input_shape = (1536,),
                          units = 512,
                          activation = "relu",
                          ),
  tf.keras.layers.Dropout(rate = 0.2),
  tf.keras.layers.Dense(units = 256,activation = "relu"),
  tf.keras.layers.Dropout(rate = 0.2),
  tf.keras.layers.Dense(units = 128,activation = "relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(units = 64,activation = "relu"),        
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(units = 32,activation = "relu"),
  # 출력으로..
  tf.keras.layers.Dense(units = 2,
                        activation = "softmax")
])
model.compile(optimizer = "Adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               786944    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [ ]:
# 우리가 이미지 파일을 중심으로 처리를 할 때에는 폴더 중심의 generator
# 3번은 특징만 추출해서 하면 되니까..
# 특징 추출하는 모델까지만 generator로 하고,,
# 뒤에는 특징만 줄었으니 메모리상으로 돌리는 것임 !!!
# 참고) train 에서 일부분을 띠어서 validation할 때는 validation_split
#       명확하게 validation set으로 할 때는
#       validation_data = (valid 문제, valid 정)
history = model.fit(train_features,train_y,
                    validation_data = (valid_features,valid_y),
                    epochs = 100, batch_size = 128)

Epoch 1/100
14/14 [==============================] - 4s 34ms/step - loss: 0.5664 - accuracy: 0.7958 - val_loss: 0.4392 - val_accuracy: 0.8503
Epoch 2/100
14/14 [==============================] - 0s 10ms/step - loss: 0.4650 - accuracy: 0.8259 - val_loss: 0.4594 - val_accuracy: 0.8503
Epoch 3/100
14/14 [==============================] - 0s 9ms/step - loss: 0.4491 - accuracy: 0.8404 - val_loss: 0.4616 - val_accuracy: 0.8503
Epoch 4/100
14/14 [==============================] - 0s 9ms/step - loss: 0.4107 - accuracy: 0.8504 - val_loss: 0.4509 - val_accuracy: 0.8503
Epoch 5/100
14/14 [==============================] - 0s 9ms/step - loss: 0.3874 - accuracy: 0.8477 - val_loss: 0.4736 - val_accuracy: 0.8273
Epoch 6/100
14/14 [==============================] - 0s 12ms/step - loss: 0.3706 - accuracy: 0.8510 - val_loss: 0.4467 - val_accuracy: 0.8372
Epoch 7/100
14/14 [==============================] - 0s 10ms/step - loss: 0.3752 - accuracy: 0.8488 - val_loss: 0.4374 - val_accuracy: 0.8503
Epoch 8/1

In [ ]:
import cv2
test_list = os.listdir("/content/drive/MyDrive/DeepLearning/Project/test/")
result = []
for img in test_list:
  image = cv2.imread(f"/content/drive/MyDrive/DeepLearning/Project/test/{img}")
  imageHeight, imageWidth, _ = image.shape

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
  image_resized = cv2.resize(image, (150,200), interpolation=cv2.INTER_AREA)\

  test = image_resized/255.0
  test = test.reshape(-1,200,150,3)
  test = feature_model.predict(test)
  result.append(model.predict(test))
result

1/1 [==============================] - 0s 26ms/step


[array([[9.9999964e-01, 3.3305844e-07]], dtype=float32),
 array([[1.0000000e+00, 6.2064897e-12]], dtype=float32),
 array([[9.9974626e-01, 2.5377021e-04]], dtype=float32),
 array([[1.0000000e+00, 1.4599537e-11]], dtype=float32),
 array([[1.000000e+00, 9.049852e-09]], dtype=float32),
 array([[0.8165233 , 0.18347666]], dtype=float32),
 array([[9.9998343e-01, 1.6612865e-05]], dtype=float32),
 array([[1.0000000e+00, 7.8033986e-12]], dtype=float32),
 array([[9.9999678e-01, 3.2516773e-06]], dtype=float32),
 array([[9.9999988e-01, 6.0351304e-08]], dtype=float32),
 array([[1.0000000e+00, 1.5573387e-14]], dtype=float32),
 array([[0.9871515 , 0.01284849]], dtype=float32),
 array([[1.0000000e+00, 1.7683939e-12]], dtype=float32),
 array([[0.9505423 , 0.04945768]], dtype=float32),
 array([[9.9999988e-01, 1.0556024e-07]], dtype=float32),
 array([[1.000000e+00, 5.488974e-08]], dtype=float32),
 array([[0.9979254 , 0.00207456]], dtype=float32),
 array([[9.999943e-01, 5.700205e-06]], dtype=float32),
 arr

In [ ]:
import shutil
import cv2

test_path = "/content/drive/MyDrive/DeepLearning/Project/test/"
pass_path = "/content/drive/MyDrive/DeepLearning/Project/model_pass/"
fail_path = "/content/drive/MyDrive/DeepLearning/Project/model_fail/"

test_list = os.listdir(test_path)
for i,img in enumerate(test_list):
  image = cv2.imread(f"/content/drive/MyDrive/DeepLearning/Project/test/{img}")
  imageHeight, imageWidth, _ = image.shape

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
  image_resized = cv2.resize(image, (150,200), interpolation=cv2.INTER_AREA)\

  test = image_resized/255.0
  test = test.reshape(-1,200,150,3)
  test = feature_model.predict(test)
  test = model.predict(test)
  if test[0][0] > test[0][1]:
    shutil.copy(test_path + str(test_list[i]),
              fail_path)
    print(str(test_list[i]) + " ============> fail")
  else:
    shutil.copy(test_path + str(test_list[i]),
              pass_path)
    print(str(test_list[i]) + " ============> pass")
shutil.rmtree (test_path)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 35ms/step
1076397_24557146_0.jpg ============> fail
1/1 [==============================] - 0s 26ms/step
1053414_30455462_0.jpg ============> fail
1/1 [==============================] - 0s 25ms/step
1053414_30602740_0.jpg ============> fail
1/1 [==============================] - 0s 36ms/step
1053414_31707740_0.jpg ============> fail
1/1 [==============================] - 0s 17ms/step
1053414_29614406_0.jpg ============> fail
1/1 [==============================] - 0s 17ms/step
1053413_34230981_0.jpg ============> fail
1/1 [==============================] - 0s 19ms/step
1053413_34195364_0.jpg ============> fail
1/1 [==============================] - 0s 21ms/step
1053413_33818799_1.jpg ============> fail
1/1 [==============================] - 0s 25ms/step
1053413_33818799_0.jpg ============> fail
1/1 [==============================] - 0s 19ms/step
1092199_25638354_0.jpg ============> fail
1/1 [===================

In [ ]:
import os

test_path = "/content/drive/MyDrive/DeepLearning/Project/test/"
pass_path = "/content/drive/MyDrive/DeepLearning/Project/model_pass/"
fail_path = "/content/drive/MyDrive/DeepLearning/Project/model_fail/"
test_list = os.listdir(test_path)
len(test_list)

2277

In [17]:
# 특징 추출기로 : Inception V3 
import tensorflow as tf
import tensorflow_hub as hub
# 이미지생성용으로...
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

train_data_sub_folder = "/content/drive/MyDrive/DeepLearning/Project/image_classification/"
batch_size = 32
image_height = 200
image_width = 150

inception_url = "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5"
feature_model = tf.keras.Sequential(
    [
        hub.KerasLayer(inception_url, output_shape=(1536,),
                       trainable=False)
    ]
)
feature_model.build( [ None, 299,299,3])

train_datagen = ImageDataGenerator(
    rescale = 1./255.,
	validation_split = 0.25
)
valid_datagen = ImageDataGenerator(
    rescale = 1./255.,
    validation_split = 0.25
)
train_generator = train_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "training",
    batch_size = 128,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)
valid_generator = valid_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "validation",
    batch_size = 32,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)

import numpy as np

batch_step = int(9397*1) // 128
# train 데이터를 특징을 추출한 것을 메모리에 담으려고 함
train_features = []
train_y = []
# 너무 많아서 줄이기

# for idx in range(20):
for idx in range(batch_step):
    if idx % 200 == 0:
        print(idx)
    x,y = train_generator.next() # 데이터 돌리기..
    train_y.extend(y) # 정답지 추가
    features = feature_model.predict(x)
    train_features.extend(features)
train_features = np.array(train_features)
train_y = np.array(train_y)
print(train_features.shape)

batch_step = int(3131*1) // 32
# train 데이터를 특징을 추출한 것을 메모리에 담으려고 함
valid_features = []
valid_y = []

# for idx in range(5):
for idx in range(batch_step):
    if idx % 200 == 0:
        print(idx)
    x,y = valid_generator.next() # 데이터 돌리기..
    valid_y.extend(y) # 정답지 추가
    features = feature_model.predict(x)
    valid_features.extend(features)
valid_features = np.array(valid_features)
valid_y = np.array(valid_y)
print(valid_features.shape)


# # DNN 모델
# model = tf.keras.Sequential([
    
#     tf.keras.layers.Dense(input_shape = (1536,),
#                           units = 512,
#                           activation = "relu",
#                           ),
#   tf.keras.layers.Dropout(rate = 0.2),
#   tf.keras.layers.Dense(units = 256,activation = "relu"),
#   tf.keras.layers.Dropout(rate = 0.2),
#   tf.keras.layers.Dense(units = 128,activation = "relu"),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(units = 64,activation = "relu"),        
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(units = 32,activation = "relu"),
#   # 출력으로..
#   tf.keras.layers.Dense(units = 2,
#                         activation = "softmax")
# ])
# model.compile(optimizer = "Adam",
#               loss = "categorical_crossentropy",
#               metrics = ["accuracy"])

# history = model.fit(train_features,train_y,
#                     validation_data = (valid_features,valid_y),
#                     epochs = 100, batch_size = 128)

Found 11036 images belonging to 2 classes.
Found 3677 images belonging to 2 classes.
0
4/4 [==============================] - 0s 108ms/step
(9344, 1536)
0
1/1 [==============================] - 0s 48ms/step
(3104, 1536)


In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.0 MB/s eta 0:00:00


In [12]:
# colab에서 설치 후에 런타임 재실행 안해도 됨 !!
import optuna
import time
import tensorflow as tf
import PIL
from tensorflow.keras.layers import Dense, Dropout

In [21]:
# Step1 : optuna를 하기 위해서 주어진 파라미터에 맞는 모델 구조를 생성하는 함수가 필요
# 입력 : 1번 Conv layer에서 몇 개 conv를 할지..(2,3..) + 개수
#        2번 Conv layer에서 몇 개 conv을 할지..(2,3..) + 개수
#        3번 Dropout에 대한 rate를 test하고 싶다..
#        4번 분류단의 Dense layer의 node 수 조절..
# +++    추가적으로 여러가지 af, padding, etc.....

####  point가 뭔가 값을 바라보고 그 값을 최적화 시키는 것!!!! --> output 이 뭔 값!!!
## accracu, mse
def create_model(input_num_node,dnn_dropout_rate, dnn_num_node ,num_dl_layers):
  # 여기서는 리스트 방식이 아니라 append 방식으로 설계를 하겠음. --> 앞에서부터
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(input_shape = (1536,),
                          units = input_num_node,
                          activation = "relu",
                          ))
  for i in range(num_dl_layers):
    model.add( Dense(units = dnn_num_node, activation = "relu"))
    model.add( Dropout(rate = dnn_dropout_rate))
  # model.add( Dense(units = dnn_num_node, activation = "relu"))
  # model.add( Dropout(rate = dnn_dropout_rate))
  # model.add( Dense(units = dnn_num_node, activation = "relu"))
  # model.add( Dropout(rate = dnn_dropout_rate))
  # model.add( Dense(units = dnn_num_node, activation = "relu"))
  # model.add( Dropout(rate = dnn_dropout_rate))
  model.add( Dense(units = 32, activation = "relu"))
  model.add( Dropout(rate = dnn_dropout_rate))
  model.add( Dense(units = 2, activation = "softmax"))
  return model

In [22]:

# Step2 : 최적의 값을 찾을 함수 Object function
def object_res( trail ):
  # 혹시 keras에서 뒤에 백에서 홀딩하는 정보들이 있으면 처리..
  tf.keras.backend.clear_session()

  # 이 값을 변경하면서,, 모델의 성능을 보고 싶은데,,
  # 이 값에 따른 모델을 불러와야 함 !!
  input_num_node = trail.suggest_int("input_num_node",512,1024)
  dnn_dropout_rate = trail.suggest_float("dnn_dropout_rate", 0.1, 0.5)
  dnn_num_node = trail.suggest_int("dnn_num_node", 64, 512)
  # + 모델을 학습하는 영향을 미치는 파라미터도 Try **** leraning_rate!!!!! adddd
  optimizer = trail.suggest_categorical("optimizer",["adam","sgd"])
  num_dl_layers = trail.suggest_int("trail.suggest_int", 1,5)

  model = create_model(input_num_node,dnn_dropout_rate, dnn_num_node,num_dl_layers)
  model.compile(optimizer = optimizer,
                loss = "categorical_crossentropy",
                metrics = tf.keras.metrics.Precision()
                )
  # 시간 관계상 줄이기 위해서 epochs는 5로
  # + 실질적으로는 callback에서 Earlystopping을 세팅 !!!
  # +++++++ early stoppping 추가!!!
  history = model.fit(train_features,train_y,
                    validation_data = (valid_features,valid_y),
                    epochs = 100, batch_size = 128)
  score = history.history["val_accuracy"][-1]
  return score

In [23]:
train_features.shape

(9344, 1536)

In [24]:
study_my_res = optuna.create_study(direction = "maximize")
# 원래는 n_trails을 많이 해야하는데,, 시간 관계상 4으로만 줄여서 함
study_my_res.optimize(object_res,n_trials= 10)

[I 2023-06-12 07:37:24,537] A new study created in memory with name: no-name-b3f23e77-e3a5-4473-823b-a6885cee1e6a


Epoch 1/100
73/73 [==============================] - 3s 9ms/step - loss: 0.3040 - accuracy: 0.9164 - val_loss: 0.2182 - val_accuracy: 0.9394
Epoch 2/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2386 - accuracy: 0.9356 - val_loss: 0.2069 - val_accuracy: 0.9394
Epoch 3/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2221 - accuracy: 0.9356 - val_loss: 0.1902 - val_accuracy: 0.9394
Epoch 4/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2113 - accuracy: 0.9356 - val_loss: 0.1700 - val_accuracy: 0.9394
Epoch 5/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2047 - accuracy: 0.9356 - val_loss: 0.1833 - val_accuracy: 0.9394
Epoch 6/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2032 - accuracy: 0.9356 - val_loss: 0.1698 - val_accuracy: 0.9394
Epoch 7/100
73/73 [==============================] - 0s 5ms/step - loss: 0.1948 - accuracy: 0.9356 - val_loss: 0.1694 - val_accuracy: 0.9394
Epoch 8/100
7

[I 2023-06-12 07:38:13,487] Trial 0 finished with value: 0.9336340427398682 and parameters: {'input_num_node': 751, 'dnn_dropout_rate': 0.23197624709873121, 'dnn_num_node': 112, 'optimizer': 'adam', 'trail.suggest_int': 4}. Best is trial 0 with value: 0.9336340427398682.


Epoch 1/100
73/73 [==============================] - 4s 11ms/step - loss: 0.3209 - accuracy: 0.9218 - val_loss: 0.2216 - val_accuracy: 0.9394
Epoch 2/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2434 - accuracy: 0.9356 - val_loss: 0.2504 - val_accuracy: 0.9394
Epoch 3/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2410 - accuracy: 0.9355 - val_loss: 0.2306 - val_accuracy: 0.9394
Epoch 4/100
73/73 [==============================] - 0s 7ms/step - loss: 0.2278 - accuracy: 0.9356 - val_loss: 0.2225 - val_accuracy: 0.9394
Epoch 5/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2163 - accuracy: 0.9356 - val_loss: 0.1865 - val_accuracy: 0.9394
Epoch 6/100
73/73 [==============================] - 0s 5ms/step - loss: 0.2128 - accuracy: 0.9356 - val_loss: 0.1952 - val_accuracy: 0.9394
Epoch 7/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2110 - accuracy: 0.9356 - val_loss: 0.1754 - val_accuracy: 0.9394
Epoch 8/100


[I 2023-06-12 07:39:36,744] Trial 1 finished with value: 0.9255799055099487 and parameters: {'input_num_node': 927, 'dnn_dropout_rate': 0.356577455405519, 'dnn_num_node': 75, 'optimizer': 'adam', 'trail.suggest_int': 3}. Best is trial 0 with value: 0.9336340427398682.


...

[I 2023-06-12 07:46:04,785] Trial 7 finished with value: 0.9355670213699341 and parameters: {'input_num_node': 727, 'dnn_dropout_rate': 0.20075360995762162, 'dnn_num_node': 76, 'optimizer': 'adam', 'trail.suggest_int': 4}. Best is trial 5 with value: 0.9429768323898315.


Epoch 1/100
73/73 [==============================] - 2s 9ms/step - loss: 0.2747 - accuracy: 0.9319 - val_loss: 0.2162 - val_accuracy: 0.9394
Epoch 2/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2534 - accuracy: 0.9354 - val_loss: 0.2076 - val_accuracy: 0.9394
Epoch 3/100
73/73 [==============================] - 1s 7ms/step - loss: 0.2461 - accuracy: 0.9355 - val_loss: 0.2029 - val_accuracy: 0.9394
Epoch 4/100
73/73 [==============================] - 1s 9ms/step - loss: 0.2408 - accuracy: 0.9356 - val_loss: 0.1961 - val_accuracy: 0.9394
Epoch 5/100
73/73 [==============================] - 1s 8ms/step - loss: 0.2307 - accuracy: 0.9356 - val_loss: 0.1970 - val_accuracy: 0.9394
Epoch 6/100
73/73 [==============================] - 1s 7ms/step - loss: 0.2264 - accuracy: 0.9355 - val_loss: 0.1848 - val_accuracy: 0.9394
Epoch 7/100
73/73 [==============================] - 1s 8ms/step - loss: 0.2219 - accuracy: 0.9356 - val_loss: 0.1844 - val_accuracy: 0.9394
Epoch 8/100
7

[I 2023-06-12 07:46:55,733] Trial 8 finished with value: 0.9358891844749451 and parameters: {'input_num_node': 844, 'dnn_dropout_rate': 0.25792898048410495, 'dnn_num_node': 359, 'optimizer': 'sgd', 'trail.suggest_int': 4}. Best is trial 5 with value: 0.9429768323898315.


Epoch 1/100
73/73 [==============================] - 2s 9ms/step - loss: 0.4077 - accuracy: 0.8754 - val_loss: 0.3250 - val_accuracy: 0.9394
Epoch 2/100
73/73 [==============================] - 0s 6ms/step - loss: 0.3081 - accuracy: 0.9255 - val_loss: 0.3356 - val_accuracy: 0.9394
Epoch 3/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2865 - accuracy: 0.9312 - val_loss: 0.3113 - val_accuracy: 0.9394
Epoch 4/100
73/73 [==============================] - 1s 7ms/step - loss: 0.2705 - accuracy: 0.9347 - val_loss: 0.3193 - val_accuracy: 0.9394
Epoch 5/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2639 - accuracy: 0.9346 - val_loss: 0.2878 - val_accuracy: 0.9394
Epoch 6/100
73/73 [==============================] - 0s 7ms/step - loss: 0.2554 - accuracy: 0.9349 - val_loss: 0.2852 - val_accuracy: 0.9394
Epoch 7/100
73/73 [==============================] - 0s 6ms/step - loss: 0.2515 - accuracy: 0.9351 - val_loss: 0.2700 - val_accuracy: 0.9394
Epoch 8/100
7

[I 2023-06-12 07:48:18,778] Trial 9 finished with value: 0.9336340427398682 and parameters: {'input_num_node': 541, 'dnn_dropout_rate': 0.4725184011572249, 'dnn_num_node': 228, 'optimizer': 'sgd', 'trail.suggest_int': 5}. Best is trial 5 with value: 0.9429768323898315.
